In [1]:
import pandas as pd

In [34]:
# Charger le fichier CSV
df = pd.read_csv(r"C:/Users/antoa/Desktop/projet certif/app/rag_csv.csv", encoding="utf-8")

pas de valeurs manquantes

In [35]:
df.isnull().sum()

index           0
url             0
nom_produit     0
img_produit     0
prix_produit    0
contenance      0
pg_vg           0
origine         0
frais           0
surbooste       0
saveur          0
description     0
brand           0
gout            0
info_brand      0
id_produit      0
dtype: int64

2. Tester l'intégrité des embeddings
Avant d'utiliser les embeddings (sentence-transformers/all-MiniLM-L6-v2), il faut tester s'ils génèrent bien des vecteurs.

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

def test_embeddings():
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    text = "Test embedding"
    embedding = model.encode(text)

    assert isinstance(embedding, np.ndarray), "L'embedding n'est pas un numpy array"  #embedding est bien un objet de type numpy.ndarray (un tableau numpy)
    assert embedding.shape[0] > 0, "L'embedding est vide" # La dimension du tableau (le nombre d'éléments dans l'embedding) est supérieure à 0, ce qui garantit que l'embedding n'est pas vide.
    print("Test passé : Embeddings fonctionnels.")

test_embeddings()

c:\Users\antoa\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Test passé : Embeddings fonctionnels.


In [43]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

# Charger le modèle
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Fonction pour tester les embeddings
def test_embeddings_from_csv(csv_file):
    # Charger le CSV
    df = pd.read_csv(csv_file, encoding="utf-8")

    # Supposons que la colonne contenant les textes s'appelle 'texte' ou 'content'
    if 'texte' not in df.columns and 'content' not in df.columns:
        print("Erreur : La colonne 'texte' ou 'content' est absente.")
        return

    # Extraire les textes
    texts = df['texte'] if 'texte' in df.columns else df['content']

    # Tester les embeddings pour chaque texte
    for i, text in enumerate(texts):
        embedding = model.encode(text)

        # Vérifications des assertions
        assert isinstance(embedding, np.ndarray), f"L'embedding à la ligne {i+1} n'est pas un numpy array"
        assert embedding.shape[0] > 0, f"L'embedding à la ligne {i+1} est vide"
        print(f"Test passé pour la ligne {i+1}: Embedding fonctionnel.")

# Appeler la fonction pour tester le CSV
test_embeddings_from_csv(r"C:/Users/antoa/Desktop/projet certif/app/rag_csv.csv")
# df = pd.read_csv(r"C:/Users/antoa/Desktop/projet certif/app/rag_csv.csv", encoding="utf-8")
# ../RGBD/table_produits/produits.csv

Erreur : La colonne 'texte' ou 'content' est absente.


In [ ]:
from config import Config
from langchain_community.vectorstores import FAISS

from app.embeddings import embeddings

In [48]:
import sys
sys.path.append(r'C:\Users\antoa\Desktop\projet certif')

In [60]:
from langchain.document_loaders import CSVLoader
from config import Config
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import numpy as np



In [52]:
# Configuration des embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
loader = CSVLoader(file_path=Config.CSV_FILE_PATH, encoding='utf-8')


In [53]:
# 1. Chargement des documents
def load_documents(file_path):
    try:
        loader = CSVLoader(file_path=Config.CSV_FILE_PATH, encoding='utf-8')
        documents = loader.load()
        print(f"Nombre de documents chargés : {len(documents)}")
        return documents
    except Exception as e:
        print(f"Erreur lors du chargement des documents : {e}")
        return []

In [62]:
# 2. Création et sauvegarde du vecteur
def save_vectorstore(documents):
    try:
        vectorstore = FAISS.from_documents(documents, embeddings)
        vectorstore.save_local('faiss_vector_store')
        print("Vectorstore sauvegardé avec succès.")
        return vectorstore
    except Exception as e:
        print(f"Erreur lors de la création du vectorstore : {e}")
        return None

In [63]:
# 3. Chargement du vecteur
def load_vectorstore():
    try:
        vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)
        print("Vectorstore chargé avec succès.")
        return vectorstore
    except Exception as e:
        print(f"Erreur lors du chargement du vectorstore : {e}")
        return None

In [64]:
# 4. Tester la recherche dans le vectorstore
def test_search_in_vectorstore(query):
    try:
        vectorstore = load_vectorstore()
        if vectorstore:
            retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.1})
            results = retriever.retrieve(query)
            print(f"Résultats de la recherche : {len(results)} document(s) trouvé(s).")
            return results
        else:
            print("Le vectorstore n'a pas pu être chargé.")
            return None
    except Exception as e:
        print(f"Erreur lors de la recherche : {e}")
        return None

In [69]:
def test_search_in_vectorstore(query):
    try:
        vectorstore = load_vectorstore()
        if vectorstore:
            retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.1})
            # Utilisation de la méthode 'get_relevant_documents' pour récupérer les documents
            results = retriever.get_relevant_documents(query)  # Changer de 'retrieve' à 'get_relevant_documents'
            print(f"Résultats de la recherche : {len(results)} document(s) trouvé(s).")
            return results
        else:
            print("Le vectorstore n'a pas pu être chargé.")
            return None
    except Exception as e:
        print(f"Erreur lors de la recherche : {e}")
        return None


In [70]:
# Processus complet de test
def test_vectorstore(file_path, query):
    documents = load_documents(file_path)  # 1. Charger les documents
    if documents:
        save_vectorstore(documents)  # 2. Sauvegarder le vectorstore
        test_search_in_vectorstore(query)  # 4. Tester la recherche


In [71]:
# Tester avec ton fichier CSV et une requête d'exemple
file_path = 'rag_csv.csv'
query = "Exemple de requête pour rechercher un document"
test_vectorstore(file_path, query)

Nombre de documents chargés : 100
Vectorstore sauvegardé avec succès.
Vectorstore chargé avec succès.
Résultats de la recherche : 4 document(s) trouvé(s).


C:\Users\antoa\AppData\Local\Temp\ipykernel_18456\3343200202.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)  # Changer de 'retrieve' à 'get_relevant_documents'
